In [72]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize
import time ## REMOVE THIS LATER

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.

CHECK DELIST:
- Purpose: checks that there is actually valid market data for this stock
- Code: if 'Close' not in data.columns or data['Close'].isnull().all():

PORTFOLIO GENERATOR
- Purpose: calculate how many shares of each stock to buy, and how much to pay in fees
- Code:

portfolio_value = 1000000 * weights  # Portfolio value based on optimized weights

shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers])  # Number of shares

fee = np.minimum(3.95, 0.001 * shares)

portfolio_val -= fee


In [73]:
#October 01, 2023 to September 30, 2024, 2023

START_DATE = '2023-10-01'
END_DATE = '2024-09-30'

 # date on which we will take stock Close prices
TARGET_DATE = '2024-11-18'

INVESTMENT = 1000000

In [74]:
# filter out invalid stocks
def validity(tickers):

    # list to keep all of t
    # he valid stocks
    final_list = []

    for ticker in tickers:
        #only append the stock to the final stock list if it is isn't delisted, has a US or CAD currency, and has the required monthly share volume
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker)):
                final_list.append(ticker)
    return final_list

# checks if the consumed Ticker is delisted
def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        time.sleep(0.3)
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock
            if 'Close' not in data.columns or data['Close'].isnull().all():
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

# checks if the consumed Ticker meets the requirement of an average monthly share volume of 100,000 shares
def check_volume(ticker):
    volume = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Volume']
    time.sleep(0.3)
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

# checks whether or not the consumed Ticker is listed in CAD or in USD
def check_currency(ticker):
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # if there's an error retrieving the data, we can assume it doesnt satisfy the required monthly volume

# read in the provided Ticker file
all_data = pd.read_csv('Tickers_Example.csv', header=None)

# create a new DataFrame only containing the valid Tickers in the consumed CSV
all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)



$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


KeyboardInterrupt: 

In [ ]:
# create a DataFrame to store 
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])

# calculate and store all returns for all all Tickers
for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        time.sleep(0.3)
        ticker_returns[ticker] = data.pct_change().dropna()  # this code calculates daily returns
        
ticker_returns.index = pd.to_datetime(ticker_returns.index.strftime("%Y-%m-%d"))

ticker_returns

KeyboardInterrupt: 

In [ ]:
def std_rank(tickers):
    std_devs = ticker_returns.std()
    
    # Create a DataFrame with tickers and their respective standard deviations
    ranked_stocks = pd.DataFrame({
        "ticker": std_devs.index,
        "STD": std_devs.values
    })
    
    # Sort by standard deviation in ascending order (lowest STD at the top)
    ranked_stocks = ranked_stocks.sort_values(by="STD", ascending=True)
    
    return ranked_stocks['ticker'].values[:24].tolist()


top_24 = std_rank(ticker_returns)

In [ ]:
# fetch TSX60 index information
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d')
time.sleep(0.3)

# fetch SP500 index information
sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d')

# reindex sp500 so that it shares the same index as tsx60
sp500 = sp500.reindex(tsx60.index, method='nearest')

# calculate the average market return
market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_returns.index = pd.to_datetime(market_returns.index.strftime('%Y-%m-%d'))
market_returns = market_returns.dropna()
# market_returns_mean = market_returns.mean()

# store the average market returns in a DataFrame
market_returns_df = pd.DataFrame(market_returns)

# display(market_returns_df.index)

In [ ]:
common_dates = market_returns_df.index.intersection(ticker_returns.index)

market_returns_df = market_returns_df.loc[common_dates]
ticker_returns = ticker_returns.loc[common_dates]

months = market_returns_df.resample('MS').bfill().index

for month in months:
    temp = market_returns_df[(market_returns_df.index.month == month.month) & (market_returns_df.index.year == month.year)].index
    if len(temp) < 18:
        market_returns_df.drop(index=temp, inplace=True)
        ticker_returns.drop(index=temp, inplace=True)

# print([val for val in ticker_returns.index if val not in market_returns_df.index])

# print(len(market_returns_df))
# print(len(ticker_returns))

In [ ]:

ticker_lst = top_24
print(ticker_lst)
index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start="2024-11-20", end="2024-11-21")["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker):
    listed_currency = ticker.fast_info["currency"]
    if listed_currency == "CAD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
    if listed_currency == "USD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"] * (1/exch_rate)
    else: print("ticker currency is not in USD or CAD error")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst)

[*********************100%***********************]  1 of 1 completed

['KO', 'RY.TO', 'CL', 'PG', 'T.TO', 'PM', 'TD.TO', 'LMT', 'PEP', 'UNP', 'BK', 'MO', 'ABT', 'ABBV', 'BLK', 'AIG', 'MRK', 'UNH', 'AAPL', 'ACN', 'AXP', 'BAC', 'PFE', 'C']


In [ ]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)

display(stock_returns)

,XIU.TO,^GSPC,Market Returns,KO,RY.TO,CL,PG,T.TO,PM,TD.TO,...,BLK,AIG,MRK,UNH,AAPL,ACN,AXP,BAC,PFE,C
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03 00:00:00-04:00,-0.008913,-0.013744,-0.011329,-0.010815,-0.024235,-0.004098,0.002066,0.006083,-0.012471,-0.009452,...,-0.027323,-0.023481,-0.007606,-0.009911,-0.007770,-0.006585,-0.029636,-0.029588,-0.001178,-0.015802
2023-10-04 00:00:00-04:00,0.001384,0.008110,0.004747,0.002915,0.009281,0.001277,0.003299,0.032093,0.011743,-0.002637,...,0.017683,0.012278,0.003930,0.002257,0.007309,0.010319,0.010893,0.001158,-0.017404,-0.001254
2023-10-05 00:00:00-04:00,0.005872,-0.001304,0.002284,-0.048328,0.007706,-0.018282,-0.014866,0.019378,0.007117,0.011595,...,0.005923,0.013309,0.013996,0.010987,0.007198,0.000873,0.007259,0.004626,0.004803,0.001758
2023-10-06 00:00:00-04:00,0.004464,0.011815,0.008140,0.014509,-0.001477,0.002310,0.002017,-0.003095,0.002066,0.005164,...,0.012432,0.011970,0.002703,0.016621,0.014751,0.008170,0.015161,0.000384,-0.010159,0.017302
2023-10-10 00:00:00-04:00,0.014359,0.011545,0.012952,0.016748,0.005309,0.000864,0.004789,0.005322,0.020397,0.009022,...,-0.009734,0.004929,-0.002599,-0.001086,0.005071,0.000417,0.010782,0.036057,0.001207,0.021444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-23 00:00:00-04:00,0.001934,0.002809,0.002371,0.001256,0.006413,0.002426,-0.002583,-0.010110,0.002484,-0.014163,...,0.009496,0.007321,-0.013143,-0.002765,-0.007581,0.008893,-0.006283,-0.009933,-0.003739,0.007081
2024-09-24 00:00:00-04:00,0.000276,0.002511,0.001393,-0.005576,-0.001022,0.004646,-0.003050,-0.004885,0.000991,-0.006604,...,0.003861,-0.011844,-0.005794,0.003104,0.003974,0.001209,-0.004078,-0.010534,0.006482,-0.019335
2024-09-25 00:00:00-04:00,-0.001929,-0.001861,-0.001895,0.001682,0.003069,-0.000385,-0.005657,-0.007140,0.003960,-0.010730,...,-0.004707,-0.005448,-0.002001,0.001947,-0.004398,-0.007567,-0.000150,-0.005070,-0.019322,-0.016132


In [ ]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


,KO,RY.TO,CL,PG,T.TO,PM,TD.TO,LMT,PEP,UNP,BK,MO,ABT,ABBV,BLK,AIG,MRK,UNH,AAPL,ACN,AXP,BAC,PFE,C
KO,1.000000,0.150028,0.551266,0.488969,0.236725,0.497467,0.064342,0.387014,0.646894,0.285296,0.219946,0.471245,0.318918,0.371829,0.178112,0.071416,0.143790,0.319145,0.026443,0.187600,0.099315,0.151610,0.254702,0.120692
RY.TO,0.150028,1.000000,0.026979,0.012954,0.345413,0.221040,0.411744,0.130602,0.102936,0.421009,0.493331,0.192222,0.115915,0.171634,0.524025,0.446861,0.057772,0.083948,0.150188,0.162999,0.447849,0.533827,0.287793,0.462149
CL,0.551266,0.026979,1.000000,0.650921,0.113675,0.290320,-0.052182,0.163075,0.492162,0.169248,0.062845,0.364368,0.287081,0.255105,0.076371,0.093821,0.237790,0.270430,-0.116012,0.060829,0.043177,0.015824,0.134659,0.034198
PG,0.488969,0.012954,0.650921,1.000000,0.190890,0.346123,-0.071849,0.167920,0.496513,0.073393,0.026912,0.351489,0.326467,0.308909,-0.004059,0.002485,0.352642,0.235571,-0.085385,0.080111,-0.042149,0.032634,0.077694,-0.032105
T.TO,0.236725,0.345413,0.113675,0.190890,1.000000,0.342204,0.253683,0.130235,0.195588,0.247468,0.244922,0.269787,0.186949,0.200059,0.274397,0.142083,0.133042,0.194793,0.087044,0.255145,0.092741,0.229975,0.208743,0.191228
PM,0.497467,0.221040,0.290320,0.346123,0.342204,1.000000,0.206223,0.336054,0.348122,0.168386,0.282535,0.535202,0.336772,0.326515,0.240839,0.160819,0.209370,0.242959,0.048644,0.254522,0.167529,0.283917,0.324761,0.187984
TD.TO,0.064342,0.411744,-0.052182,-0.071849,0.253683,0.206223,1.000000,0.133259,0.035940,0.331002,0.411828,0.165993,0.069380,0.084723,0.420355,0.291964,0.077174,0.102927,0.071867,0.195094,0.431859,0.505171,0.268124,0.475708
LMT,0.387014,0.130602,0.163075,0.167920,0.130235,0.336054,0.133259,1.000000,0.305644,0.208103,0.132774,0.224215,0.255473,0.182828,0.030874,0.212449,-0.010278,0.173642,-0.040205,0.020030,0.118069,0.160831,0.142096,0.116521
PEP,0.646894,0.102936,0.492162,0.496513,0.195588,0.348122,0.035940,0.305644,1.000000,0.138857,0.164880,0.362299,0.236716,0.291194,0.070474,0.037019,0.149335,0.298206,0.010972,0.106314,0.064289,0.133348,0.142138,0.105849
UNP,0.285296,0.421009,0.169248,0.073393,0.247468,0.168386,0.331002,0.208103,0.138857,1.000000,0.351441,0.253094,0.225640,0.224117,0.497943,0.299557,0.109538,0.167154,0.110929,0.258009,0.317118,0.405935,0.143403,0.334470


In [ ]:


#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)
print(stock_pairs)

[['CL', 'AAPL'], ['PG', 'TD.TO'], ['BK', 'UNH'], ['LMT', 'MRK'], ['ABT', 'AIG'], ['PEP', 'AXP'], ['KO', 'C'], ['ACN', 'PFE'], ['PM', 'UNP'], ['RY.TO', 'ABBV'], ['MO', 'BAC'], ['T.TO', 'BLK']]


In [ ]:
#Arbitrarily assign a seed to keep results consistent
random.seed(21152764)

#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    for i in range(6, len(stock_pairs)):
        #simple iteration of 5 times
        for j in range(0,5):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)-1), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

port_list = make_port_list(stock_pairs)
display(port_list)
len(port_list)

[['BK',
  'UNH',
  'PEP',
  'AXP',
  'ABT',
  'AIG',
  'ACN',
  'PFE',
  'PM',
  'UNP',
  'LMT',
  'MRK'],
 ['LMT',
  'MRK',
  'PG',
  'TD.TO',
  'PEP',
  'AXP',
  'RY.TO',
  'ABBV',
  'PM',
  'UNP',
  'KO',
  'C'],
 ['PG',
  'TD.TO',
  'KO',
  'C',
  'PM',
  'UNP',
  'BK',
  'UNH',
  'ACN',
  'PFE',
  'ABT',
  'AIG'],
 ['MO',
  'BAC',
  'KO',
  'C',
  'RY.TO',
  'ABBV',
  'BK',
  'UNH',
  'PG',
  'TD.TO',
  'PM',
  'UNP'],
 ['KO',
  'C',
  'PM',
  'UNP',
  'ABT',
  'AIG',
  'RY.TO',
  'ABBV',
  'LMT',
  'MRK',
  'ACN',
  'PFE'],
 ['PM',
  'UNP',
  'RY.TO',
  'ABBV',
  'PEP',
  'AXP',
  'MO',
  'BAC',
  'PG',
  'TD.TO',
  'BK',
  'UNH',
  'ACN',
  'PFE'],
 ['ACN',
  'PFE',
  'PM',
  'UNP',
  'MO',
  'BAC',
  'CL',
  'AAPL',
  'LMT',
  'MRK',
  'RY.TO',
  'ABBV',
  'KO',
  'C'],
 ['ABT',
  'AIG',
  'PM',
  'UNP',
  'ACN',
  'PFE',
  'CL',
  'AAPL',
  'PG',
  'TD.TO',
  'BK',
  'UNH',
  'PEP',
  'AXP'],
 ['PG',
  'TD.TO',
  'LMT',
  'MRK',
  'KO',
  'C',
  'PM',
  'UNP',
  'RY.TO',
  'AB

[['BK',
  'UNH',
  'PEP',
  'AXP',
  'ABT',
  'AIG',
  'ACN',
  'PFE',
  'PM',
  'UNP',
  'LMT',
  'MRK'],
 ['LMT',
  'MRK',
  'PG',
  'TD.TO',
  'PEP',
  'AXP',
  'RY.TO',
  'ABBV',
  'PM',
  'UNP',
  'KO',
  'C'],
 ['PG',
  'TD.TO',
  'KO',
  'C',
  'PM',
  'UNP',
  'BK',
  'UNH',
  'ACN',
  'PFE',
  'ABT',
  'AIG'],
 ['MO',
  'BAC',
  'KO',
  'C',
  'RY.TO',
  'ABBV',
  'BK',
  'UNH',
  'PG',
  'TD.TO',
  'PM',
  'UNP'],
 ['KO',
  'C',
  'PM',
  'UNP',
  'ABT',
  'AIG',
  'RY.TO',
  'ABBV',
  'LMT',
  'MRK',
  'ACN',
  'PFE'],
 ['PM',
  'UNP',
  'RY.TO',
  'ABBV',
  'PEP',
  'AXP',
  'MO',
  'BAC',
  'PG',
  'TD.TO',
  'BK',
  'UNH',
  'ACN',
  'PFE'],
 ['ACN',
  'PFE',
  'PM',
  'UNP',
  'MO',
  'BAC',
  'CL',
  'AAPL',
  'LMT',
  'MRK',
  'RY.TO',
  'ABBV',
  'KO',
  'C'],
 ['ABT',
  'AIG',
  'PM',
  'UNP',
  'ACN',
  'PFE',
  'CL',
  'AAPL',
  'PG',
  'TD.TO',
  'BK',
  'UNH',
  'PEP',
  'AXP'],
 ['PG',
  'TD.TO',
  'LMT',
  'MRK',
  'KO',
  'C',
  'PM',
  'UNP',
  'RY.TO',
  'AB

In [ ]:
portfolio_data = []
def portfolio_generator(port_list):
    global portfolio_data
   
    # this dictionary will store Ticker closing prices on the target date
    ticker_prices = {} 
     
    # populate ticker_prices
    for ticker in top_24:
        stock = yf.Ticker(ticker)
        history = stock.history(start=TARGET_DATE, end='2024-11-19')
        time.sleep(0.3)  # Add a small delay to avoid rate limiting
        
        # Fetch the closing price for the target date
        if not history.empty and TARGET_DATE in history.index.strftime('%Y-%m-%d'):
            close_price = history.loc[history.index.strftime('%Y-%m-%d') == TARGET_DATE, 'Close'].values[0]
            ticker_prices[ticker] = close_price
        else:
            ticker_prices[ticker] = None  # No data available for that date

    #WE NEED DF OF TICKER PRICES
    
    # creating portfolios -- starting wtih 12 stocks, going upto 24 stocks
    for lst in port_list:

        # for the ith portfolio, select the first i tickers
        selected_tickers = lst
        
        # objective function -- will be used in optimization steps
        # returns the tracking error between the portfolio's returns and the market's returns
        def objective(weights):
            return np.std(np.dot(ticker_returns[selected_tickers].dropna().values, weights) - market_returns_df.dropna().values)
        
        # initial weights are calcualted so that, to start, every stock has equal weight
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # this constraint ensures that all of the stock weights sum to 1 (100%)
        def sum_weights(weights):
            return sum(weights) - 1
        
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # the bounds for each stock are that each stock must have a minimum weight of 1/2n% and maximum weight of 15%
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # optimize the portfolio weights
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        if result.success:

            weights = result.x
                        
            # calculate portfolio value and the number of shares to purchase
            portfolio_value = 1000000 * weights
            shares = portfolio_value / np.array([ticker_prices[ticker] for ticker in selected_tickers]) 

            # calculate fees
            fee = np.minimum(3.95, 0.001 * shares)
            
            # subtract fees from portfolio value
            portfolio_value -= fee
            
            # Add to the results
            portfolio_data.append({
                'tickers': ', '.join(selected_tickers),  # Tickers in the portfolio
                'weights': result.x,  # Optimized weights
                'Tracking Error': result.fun,  # Standard deviation (risk)
                # 'STD:': portfolio
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_cost': portfolio_value.sum() + sum(fee),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_value.sum(),  # Total portfolio value after fees
                'final_fees': sum(fee)
            })

    return pd.DataFrame(portfolio_data)

portfolio_df = portfolio_generator(port_list)
portfolio_df

['BK', 'UNH', 'PEP', 'AXP', 'ABT', 'AIG', 'ACN', 'PFE', 'PM', 'UNP', 'LMT', 'MRK']
['LMT', 'MRK', 'PG', 'TD.TO', 'PEP', 'AXP', 'RY.TO', 'ABBV', 'PM', 'UNP', 'KO', 'C']
['PG', 'TD.TO', 'KO', 'C', 'PM', 'UNP', 'BK', 'UNH', 'ACN', 'PFE', 'ABT', 'AIG']
['MO', 'BAC', 'KO', 'C', 'RY.TO', 'ABBV', 'BK', 'UNH', 'PG', 'TD.TO', 'PM', 'UNP']
['KO', 'C', 'PM', 'UNP', 'ABT', 'AIG', 'RY.TO', 'ABBV', 'LMT', 'MRK', 'ACN', 'PFE']
['PM', 'UNP', 'RY.TO', 'ABBV', 'PEP', 'AXP', 'MO', 'BAC', 'PG', 'TD.TO', 'BK', 'UNH', 'ACN', 'PFE']
['ACN', 'PFE', 'PM', 'UNP', 'MO', 'BAC', 'CL', 'AAPL', 'LMT', 'MRK', 'RY.TO', 'ABBV', 'KO', 'C']
['ABT', 'AIG', 'PM', 'UNP', 'ACN', 'PFE', 'CL', 'AAPL', 'PG', 'TD.TO', 'BK', 'UNH', 'PEP', 'AXP']
['PG', 'TD.TO', 'LMT', 'MRK', 'KO', 'C', 'PM', 'UNP', 'RY.TO', 'ABBV', 'ABT', 'AIG', 'BK', 'UNH']
['RY.TO', 'ABBV', 'MO', 'BAC', 'PEP', 'AXP', 'BK', 'UNH', 'PG', 'TD.TO', 'ABT', 'AIG', 'LMT', 'MRK']
['CL', 'AAPL', 'BK', 'UNH', 'PEP', 'AXP', 'KO', 'C', 'PM', 'UNP', 'RY.TO', 'ABBV', 'LMT', 

""


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

ALEX
- coded get_sector function
- edited validity function
- matched indexes of DataFrames (the first containing the market returns, and the second containing ticker returns)

DEREK
- coded correlation function

SHARUGA
- coded validity function
- coded portfolio_generator function